# DNN Saccade Classification

In [1]:
%pylab qt
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


### Loading Data

In [23]:
from typing import Tuple

from numpy import ndarray, argmax, hstack

from random import shuffle



from eyelib.datasets import load

import tensorflow as tf



dataset = load('full_saccadic')
X = dataset['X']
Y = dataset['Y']

def iterate_batches(x, y, batch_size) -> Tuple[ndarray, ndarray]:
    index = 0
    while index < len(x):
        yield x[index:index+batch_size,:], y[index:index+batch_size,:]
        index += batch_size
    
def train_test_validation_split(x: ndarray, y: ndarray, train_ratio: float=0.7, test_ratio: float=0.2) -> Tuple[ndarray, ndarray, ndarray, ndarray, ndarray, ndarray]:
    tmp = hstack((x, y))
    shuffle(tmp)
    first_split = int(len(tmp) * train_ratio)
    second_split = first_split + int(len(tmp) * test_ratio)
    train = tmp[:first_split,:]
    test = tmp[first_split:second_split,:]
    validation = tmp[second_split:,:]
    return train[:,:x.shape[1]], train[:,x.shape[1]:], test[:,:x.shape[1]], test[:,x.shape[1]:], validation[:,:x.shape[1]], validation[:,x.shape[1]:]


### Creating the Network

In [27]:
learning_rate = 0.00005
epochs = 100
batch_size = 25
dropout = 0.9

samples_count = 192

x = tf.placeholder(tf.float32, [None, samples_count])
y = tf.placeholder(tf.float32, [None, 3])

x_shaped = tf.reshape(x, [-1, 1, samples_count, 1])

def conv_layer(input_data, num_input_channels, num_filters, filter_width, pool_width, name):
    conv_filt_shape = [1, filter_width, num_input_channels, num_filters]
    
    weights = tf.Variable(
        tf.truncated_normal(conv_filt_shape, stddev=0.03), 
        name=f'{name}_W'
    )
    bias = tf.Variable(
        tf.truncated_normal([num_filters]), 
        name=f'{name}_b'
    )
    
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME') + bias
    
    out_layer = tf.nn.relu(out_layer)
    
    ksize = [1, 1, pool_width, 1]
    strides = [1, 1, pool_width, 1]
    print("Salida de la Relu " + str(out_layer.shape) + "valor de pool_width" + str(pool_width))
    out_layer = tf.nn.max_pool(value=out_layer, ksize=ksize, strides=strides, padding='SAME')
    print("Salida del maxpool " + str(out_layer.shape)+ "valor de pool_width" + str(pool_width))
    return out_layer

conv1_pool_width = 2  # 192 -> 96
conv2_pool_width = 2  # 96 -> 48

conv1 = conv_layer(input_data=x_shaped, num_input_channels=1, num_filters=16, filter_width=9,pool_width= conv1_pool_width, name='conv1')
print(conv1.shape)
conv2 = conv_layer(conv1, 16, 32, 7, conv2_pool_width, name='conv2')

pool_sizes_mul = conv1_pool_width * conv2_pool_width
print("pool_sizes_mul " + str(pool_sizes_mul))
fully_connected_samples = int(samples_count / pool_sizes_mul) * 32

flattened = tf.reshape(conv2, [-1, fully_connected_samples])


wd1 = tf.Variable(tf.truncated_normal([fully_connected_samples, 768], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([768], stddev=0.01), name='bd1')
layer1 = tf.matmul(flattened, wd1) + bd1
layer1 = tf.nn.relu(layer1)
layer1 = tf.nn.dropout(layer1, dropout)

wd2 = tf.Variable(tf.truncated_normal([768, 3], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([3], stddev=0.01), name='bd2')
layer2 = tf.matmul(layer1, wd2) + bd2
y_ = tf.nn.softmax(layer2)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=layer2, labels=y))

optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
cm = tf.confusion_matrix(tf.argmax(y, 1), tf.argmax(y_, 1), num_classes=3)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    train_x, train_y, test_x, test_y, validate_x, validate_y = train_test_validation_split(X, Y)
    for epoch in range(epochs):
        avg_cost = 0
        for batch_x, batch_y in iterate_batches(train_x, train_y, batch_size):
            _, c = sess.run([optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / (len(batch_x) / batch_size)
        
        test_acc = sess.run(accuracy, feed_dict={x: test_x, y: test_y})
        print(f"Epoch: {epoch + 1} cost = {avg_cost:.6f} test accuracy: {test_acc:.6f}")

    print("\nTraining complete!")
    print(sess.run(accuracy, feed_dict={x: validate_x, y: validate_y}))
    
    print('\nConfusion Matrix')
    print(sess.run(cm, feed_dict={x: validate_x, y: validate_y}))

Salida de la Relu (?, 1, 192, 16)valor de pool_width2
Salida del maxpool (?, 1, 96, 16)valor de pool_width2
(?, 1, 96, 16)
Salida de la Relu (?, 1, 96, 32)valor de pool_width2
Salida del maxpool (?, 1, 48, 32)valor de pool_width2
pool_sizes_mul 4
Epoch: 1 cost = 56.841024 test accuracy: 0.469537
Epoch: 2 cost = 56.354034 test accuracy: 0.454102
Epoch: 3 cost = 58.261151 test accuracy: 0.470349
Epoch: 4 cost = 57.963040 test accuracy: 0.469537
Epoch: 5 cost = 59.615530 test accuracy: 0.479285
Epoch: 6 cost = 60.794984 test accuracy: 0.480097
Epoch: 7 cost = 63.344562 test accuracy: 0.463851
Epoch: 8 cost = 64.704462 test accuracy: 0.461413
Epoch: 9 cost = 66.898454 test accuracy: 0.463851
Epoch: 10 cost = 67.709950 test accuracy: 0.464663
Epoch: 11 cost = 68.264326 test accuracy: 0.464663
Epoch: 12 cost = 68.831947 test accuracy: 0.463038
Epoch: 13 cost = 69.398125 test accuracy: 0.462226
Epoch: 14 cost = 69.262723 test accuracy: 0.462226
Epoch: 15 cost = 69.584415 test accuracy: 0.4606

In [22]:
print(wd1.shape)
print(y_.shape)


(1536, 768)
(?, 3)


In [ ]:
rd.shuffle